# **ENGG680 - Introduction to Digital Engineering**
## *Lab Assignment 4: ML Applications - Vehicle Motion State Classification*

## Preliminary: Certificate of Work


*We, the undersigned, certify that this is our own work, which has been done expressly for this course, either without the assistance of any other party or where appropriate we have acknowledged the work of others. Further, we have read and understood the section in the university calendar on plagiarism/cheating/other academic misconduct and we are aware of the implications thereof. We request that the total mark for this assignment be distributed as follows among group members:*

|          | First Name | Last Name | Signature (Full Name, Date) | Hours | Contribution % |
|----------|------------|-----------|-----------------------------|-------|----------------|
| Member 1: | Sebastian | Vanegas Perez | Sebastian Vanegas Perez, 29/11/2024 | 8 | 25 |
| Member 2: | Subhreet | Jhinger | Subhreet Jhinger, 29/11/2024 | 8 | 25 |
| Member 3: | Abdullah | Ajlan | Abdullah Ajlan, 29/11/2024 | 8 | 25 |
| Member 4: | Lisa | Brantuo | Lisa Brantuo, 29/11/2024 | 8 | 25 |


---

## Section 1 - Motion Classification Using *All Features* Derived From Inertial Sensor Data (33 Marks)

---

### 1.1 Assignment datasets import (6 Marks)

---

In [1]:
# some library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install openpyxl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/engg680_2024_fall/Lab_4/

/content/drive/MyDrive/engg680_2024_fall/Lab_4


In [4]:
!ls

2020_03_14.csv	assignment_4.ipynb  urban_sensors.csv  whampoa_sensors.csv


First load the following datasets into dataframes.
* *UrbanNav-HK_Whampoa-20210521_sensors.csv* to dataframe variable named *cv_set*. This dataset is to be used for model training and hyper parameter tuning.
* *UrbanNav-HK_TST-20210517_sensors.csv* to dataframe variable named *test_set1*. This is first independent test dataset.
* *2020-03-14-16-45-35.csv* to dataframe variable named *test_set2*. This is the second independent test dataset.

In [5]:
# (2 marks)
# TODO: Load cross validation/training dataset (UrbanNav-HK_Whampoa-20210521_sensors.csv) to 'cv_set' dataframe

# uncomment and compelete below
cv_set = pd.read_csv('/content/drive/MyDrive/engg680_2024_fall/Lab_4/whampoa_sensors.csv')

# TODO: Load UrbanNav-HK_TST-20210517_sensors.csv to 'test_set1' dataframe
test_set1 = pd.read_csv('/content/drive/MyDrive/engg680_2024_fall/Lab_4/urban_sensors.csv')

# TODO: Load 2020-03-14-16-45-35.csv to 'test_set2' dataframe
test_set2 = pd.read_csv('/content/drive/MyDrive/engg680_2024_fall/Lab_4/2020_03_14.csv')

For the *cv_set*, look at the dataframe head and info, and print the list of all columns. The *test_set1* and *test_set2* have the same schema as *cv_set*, however, have different number of rows (samples).

In [6]:
# TODO: Display first two rows of cv_set
# (1 mark)

print(cv_set.head(2))

     ax_sum    ay_sum    az_sum    gx_sum    gy_sum    gz_sum     a_sum  \
0 -0.003929 -0.093081  9.789349 -0.002649  0.001406 -0.001391  9.797418   
1 -0.000966 -0.092417  9.789095 -0.001806  0.000103 -0.001943  9.798815   

      g_sum        ax_var        ay_var  ...     g_min    ax_max    ay_max  \
0  0.022554  8.046094e-07  1.319665e-07  ...  0.000003  0.001496  0.000544   
1  0.021731  1.032619e-06  1.073067e-07  ...  0.000006  0.001865  0.000615   

     az_max    gx_max    gy_max    gz_max     a_max     g_max  motion_state  
0  0.024992  0.000110  0.000075  0.000078  0.024997  0.000187          rest  
1  0.024828  0.000124  0.000098  0.000090  0.024829  0.000144          rest  

[2 rows x 33 columns]


In [7]:
# TODO: Load the info method of cv_set
# (1 mark)

cv_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ax_sum        1534 non-null   float64
 1   ay_sum        1534 non-null   float64
 2   az_sum        1534 non-null   float64
 3   gx_sum        1534 non-null   float64
 4   gy_sum        1534 non-null   float64
 5   gz_sum        1534 non-null   float64
 6   a_sum         1534 non-null   float64
 7   g_sum         1534 non-null   float64
 8   ax_var        1534 non-null   float64
 9   ay_var        1534 non-null   float64
 10  az_var        1534 non-null   float64
 11  gx_var        1534 non-null   float64
 12  gy_var        1534 non-null   float64
 13  gz_var        1534 non-null   float64
 14  a_var         1534 non-null   float64
 15  g_var         1534 non-null   float64
 16  ax_min        1534 non-null   float64
 17  ay_min        1534 non-null   float64
 18  az_min        1534 non-null 

In [8]:
# TODO: Display the number of rows and columns  of all three datasets
# (1 mark)

print(f"'cv_set' has {cv_set.shape[0]} rows and {cv_set.shape[1]} columns")
print(f"'test_set1' has {test_set1.shape[0]} rows and {test_set1.shape[1]} columns")
print(f"'test_set2' has {test_set2.shape[0]} rows and {test_set2.shape[1]} columns")

'cv_set' has 1534 rows and 33 columns
'test_set1' has 783 rows and 33 columns
'test_set2' has 297 rows and 33 columns


In [9]:
# TODO: Display all columns in cv_set
# (1 mark)

print(list(cv_set.columns))

['ax_sum', 'ay_sum', 'az_sum', 'gx_sum', 'gy_sum', 'gz_sum', 'a_sum', 'g_sum', 'ax_var', 'ay_var', 'az_var', 'gx_var', 'gy_var', 'gz_var', 'a_var', 'g_var', 'ax_min', 'ay_min', 'az_min', 'gx_min', 'gy_min', 'gz_min', 'a_min', 'g_min', 'ax_max', 'ay_max', 'az_max', 'gx_max', 'gy_max', 'gz_max', 'a_max', 'g_max', 'motion_state']




```
# Esto tiene formato de código
```

In all three dataframes, target vector is stored in the *motion_state* column. All other columns are features.

___

### 1.2 - Grid Search with Cross Validation (18 Marks)

___

#### 1.2.1 - Extract all features and target from cv_set

___

From the *cv_set*, load the target vector stored in the *motion_state* column into a new variable *y*

In [10]:
# TODO - Load target vector into y
# (1 mark)

y = cv_set['motion_state']

From *cv_set*, load all features, that is all columns except for the target vector, in a new dataframe variable *X*

In [11]:
# TODO - Load all features from cv_set to X
# (2 marks)
X = cv_set.drop(columns=['motion_state'])

X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


___

#### 1.2.2 - Configure and Perform Grid Search with Cross-Validation

___

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV

The dataset *(X, y)*, must be split into training/validation *(X_train, y_train)* and testing *(X_test, y_test)* sets. Use the `train_test_split` function from sklearn to create the split, where  **30%** of data is used for testing. Set the `random_state=42`, and `stratify=y`.  

In [13]:
# TODO - Perform test/train split
# (1 mark)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

Initialize a `Pipeline`, consisting of a *scaler* and a *classifer*, set the default values to *StandardScaler()*, and *SVC()*, respectively.

In [14]:
# TODO - Initialize Pipeline into variable pipe
# (2 marks)

pipe = Pipeline([
    ('scaler', StandardScaler()),  # Default scaler
    ('classifier', SVC())          # Default classifier
])

Define the grid search configuration (*param_grid*) for training and hyper parameter tuning of the *support vector classifier*. Tune, the hyper-parameter *C*, over the interval `C=[0.01, 0.1, 1 ,10,100]`, *gamma*, over the interval `gamma=[0.001, 0.01, 0.1,1]`, and use the *radial basis function* as the kernel. For the *scaler* parameter, test with the `StandardScaler`, `RobustScaler`, and `None` options.

In [15]:
# TODO - Setup grid set
# (4 marks)

param_grid = {
    'scaler': [StandardScaler(), RobustScaler(), None],
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__gamma': [0.001, 0.01, 0.1, 1],
    'classifier__kernel': ['rbf']
}

In [16]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'classifier__kernel': ['rbf'],
                         'scaler': [StandardScaler(), RobustScaler(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 1.2.3 - Visualize Grid Search Results

___

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [17]:
# TODO: Print the results from the grid search
# (2 marks)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(max(grid.cv_results_['mean_train_score'])))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}

Best cross-validation train score: 1.00
Best cross-validation validation score: 0.95
Test-set score: 0.96


Display the confusion matrix and print the classification report for the trained model. Use the *(X_test, y_test)* dataset. **Hint** - You can use your code from assignment 3.

In [18]:
# TODO - Display confusion matrix and print classification report for (X_test, y_test)
# (2 marks)

from sklearn.metrics import confusion_matrix, classification_report

y_pred = grid.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

class_report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", class_report)

Confusion Matrix:
 [[153   2   0]
 [ 10 192   5]
 [  0   3  96]]

Classification Report:
               precision    recall  f1-score   support

        rest       0.94      0.99      0.96       155
    straight       0.97      0.93      0.95       207
        turn       0.95      0.97      0.96        99

    accuracy                           0.96       461
   macro avg       0.95      0.96      0.96       461
weighted avg       0.96      0.96      0.96       461



___

#### 1.2.4 - Questions

1. Comment on the fit of the model to the dataset? (2 marks)
2. In the grid search, are the intervals used for *C* and *gamma* hyperparameters suitable? Why or why not? (2 marks)

#### 1.2.4 - Answers
1. Training fit: The model fits the training data perfectly, as evidenced by the best cross-validation train score of 1.00. However, combined with a high C value (100), this can indicate overfitting.
Validation and test performance—The test validation and test scores are very high, 0.95 and 0.96, respectively, demonstrating that the model generalizes well to unseen data.

Classification report and confusion matrix: All classes show balanced performance with high precision, recall, and F1 scores. A few misclassifications (for example, five straight misclassifications as turn and ten straight misclassifications as rest) indicate that the model successfully captures the differences between motion states.

The model performs exceptionally well overall, indicating that the selected features and hyperparameters produce accurate predictions without significant underfitting.


2.  Yes, the intervals for calculating C and gamma are appropriate.

For the C interval: The range [0.01, 0.1, 10,100] allows to model to explore with varying regularization levels. With low C values that are 0.01, 0.1 allows the stronger regularization and prevents overfitting. While, the higher C values (10, 100) focus more on reducing the classification errors, which is effective in this case because the dataset seems tidy and organized. The best C i.e. 100 demonstrates the model gets benefits from less regularization which that there is minimal noise present in the dataset.

The gamma interval [0.001, 0.01, 0.1, 1] suggests that the RBF kernel influence is tested effectively. The low gamma values (0.001, 0.01) work well with many characteristics and produce smoother decision boundaries. While the higher gamma values (0.1,1) overfit the smaller datasets and create localized decision boundaries. Following the quantity and complexity of the dataset, the optimal gamma (0.01) strikes a balance between generalization and flexibility.

For the enhancement, more values (such as C = 0.5 or gamma = 0.005) could further strengthen the results for a more exhaustive search. The strong performance attained, however, suggests that the existing intervals are adequate.

___

### 1.3 - Model Evaluation on Test Datasets 1 and 2 (9 Marks)

___

#### 1.3.1 - Extract all features and targets from test datasets

Next, the model trained in 1.2 will be tested on independent datasets, that is *test_set1* and *test_set2*. <br>
The test_sets must be seperated into features (Xs) and targets (ys).

In [19]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
# (1 marks)

y1 = test_set1['motion_state']
X1 = test_set1.drop(columns=['motion_state'])

X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [20]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2

y2 = test_set2['motion_state']
X2 = test_set2.drop(columns=['motion_state'])

X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


____

#### 1.3.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [21]:
# TODO - Print test score accuracies for test_set1 and test_set2
# (1 marks)

print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.92
test_set2 score: 0.25


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [22]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
# (1 marks)

y1_pred = grid.predict(X1)

conf_matrix1 = confusion_matrix(y1, y1_pred)
print("Confusion Matrix for test_set1:\n", conf_matrix1)

class_report1 = classification_report(y1, y1_pred)
print("\nClassification Report for test_set1:\n", class_report1)

Confusion Matrix for test_set1:
 [[200  31   0]
 [  7 387  17]
 [  0   7 134]]

Classification Report for test_set1:
               precision    recall  f1-score   support

        rest       0.97      0.87      0.91       231
    straight       0.91      0.94      0.93       411
        turn       0.89      0.95      0.92       141

    accuracy                           0.92       783
   macro avg       0.92      0.92      0.92       783
weighted avg       0.92      0.92      0.92       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [23]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

y2_pred = grid.predict(X2)

conf_matrix2 = confusion_matrix(y2, y2_pred)
print("Confusion Matrix for test_set2:\n", conf_matrix2)

class_report2 = classification_report(y2, y2_pred)
print("\nClassification Report for test_set2:\n", class_report2)

Confusion Matrix for test_set2:
 [[  0   0  16]
 [  0   0 206]
 [  0   0  75]]

Classification Report for test_set2:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.00      0.00      0.00       206
        turn       0.25      1.00      0.40        75

    accuracy                           0.25       297
   macro avg       0.08      0.33      0.13       297
weighted avg       0.06      0.25      0.10       297



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


___

#### 1.3.3 - Questions

**Based on results so far**
1. Comment on the model's performances on the datasets *(X_test, y_test)*, *(X1, y1)*, *(X2, y2)*? A significant difference in performance should be observed, what is this an example of? (3 marks)
1. Provide and explain potential causes for the observed behaviour in performance? (3 marks)

#### 1.3.3 - Answers
1. (X test, y test): With an accuracy of 96%, the model indicates it distributes well as the training set.

(X1, y1): High accuracy of 92%, indicates that the model generalizes well on the unseen data from the (test_set1) dataset. However, there is a slight decrease in the values as compared to (X_test, y_test).

(X2, y2): Low accuracy of only 25% indicated that the model performed poorly. The confusion matrix and classification report demonstrate that the model fails to consider other motions by only predicting one class (turn) for the entire dataset.

Further explains it to be an example of domain mismatch, In test_set2 data distribution differs from the training and validation datasets (cv_set, test_set1).

2. The observed behaviour could be explained by several possible causes:

(1) Test_set2 might have a different underlying distribution than cv_set and test_set1, according to Data Distribution Differences (Dataset Shift). For instance: Changes in environmental variables (e.g., weather, road conditions), noise levels, or sensor calibration. Variations in the dynamics of the vehicle's motion while data was being collected. The depiction of motion states (turn, rest, and straight) is not balanced with the training data.

(2) Overfitting to Training Data: The model's high C value (100) suggests that it has less regularization, which makes it able to fit the training data quite well. However, it may become sensitive to slight changes in feature distributions, which would lessen its robustness on datasets that are not visible but have notable differences.

(3) Feature engineering is not robust enough. Although the feature set works well for cv_set and test_set1, it might not be able to capture the variability in test_set2. This suggests that the characteristics might not apply to other situations or automobiles.

(4)  Limited Investigation of Hyperparameters - The grid search concentrated on optimizing performance on cv_set, even though it investigated several configurations. A more robust model could be produced by experimenting with more scalers or kernels, for example.

(5) Class Imbalance in test_set2 - The confusion matrix in test_set2 indicates that one class (turn) is dominant. Class imbalance or a lack of training samples for comparable scenarios in the training data may be the cause of the model's incapacity to predict other classes.

Improvement Suggestions:

(1)Preprocess and Analyze test_set2 – Comparing the feature distributions in test_set2 with test_set1 and cv_set. Reduce dataset discrepancies by using preprocessing techniques (such as feature modification or normalization).

(2) Increase Diversity of Training Dataset - Add more data that records a wider variety of situations (e.g., new environments, sensors, or vehicles).

(3) Improve Feature Selection or Engineering - Apply feature selection methods (like PCA) or develop additional features specifically suited to the variability of test_set2.

(4) Regularize the Model- To improve generalization and robustness, try with different kernels (such as polynomial or sigmoid) or use lower C values. Balance the Datasets.

___

## Section 2 - Classification Using *Selected Features* Derived From Inertial Sensor Data (12 Marks)

In this second section, an *SVC* model will be trained and tuned using grid search, however, only *selected features* will be used.

___

### 2.1 Grid Search with Cross Validation (5 Marks)

___

#### 2.1.1 Extract *selected* features and target from *cv_set*

From the *cv_set* dataframe, extract the selected features, listed below, and load them in a new dataframe *X*.
* *ax_var* - variance of accelerometer readings in x
* *ay_var* - variance of accelerometer readings in y
* *az_sum* - sum of accelerometer z readings
* *a_sum* - sum of the norm of accelerometer x, y, z readings
* *gz_var* - variance of gyroscope reading in z

In [24]:
# TODO - Load selected features from cv_set to X
# (1 marks)
X = cv_set[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]

X.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,6.983481e-10


In [25]:
# TODO - Load motion_state column (target vector) into y
y = cv_set['motion_state']

___

#### 2.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the selected feature set.

In [26]:
# TODO - Perform model training and tuning using `GridSearchCV`

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Define the pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),  # Default scaler
    ('classifier', SVC())          # Default classifier
])

# Define the parameter grid for grid search
param_grid = {
    'scaler': [StandardScaler(), RobustScaler(), None],  # Scalers to test
    'classifier__C': [0.01, 0.1, 1, 10, 100],           # C hyperparameter values
    'classifier__gamma': [0.001, 0.01, 0.1, 1],         # gamma hyperparameter values
    'classifier__kernel': ['rbf']                       # Radial basis function kernel
}

# Perform the grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

# Print the results of the grid search
print("Best parameters:", grid.best_params_)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))

Best parameters: {'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}
Best cross-validation score: 0.82
Test set score: 0.83


___

#### 2.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [27]:
# TODO: Print the results from the grid search

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(max(grid.cv_results_['mean_train_score'])))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'scaler': RobustScaler()}

Best cross-validation train score: 0.97
Best cross-validation validation score: 0.82
Test-set score: 0.83


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [28]:
# TODO - Display confusion matrix and print classification reports

from sklearn.metrics import confusion_matrix, classification_report

# Predict on the test set
y_pred = grid.predict(X_test)

# Display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Print classification report
class_report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", class_report)

Confusion Matrix:
 [[154   1   0]
 [ 20 173  14]
 [  1  44  54]]

Classification Report:
               precision    recall  f1-score   support

        rest       0.88      0.99      0.93       155
    straight       0.79      0.84      0.81       207
        turn       0.79      0.55      0.65        99

    accuracy                           0.83       461
   macro avg       0.82      0.79      0.80       461
weighted avg       0.82      0.83      0.82       461



___

#### 2.1.4 - Question


1. Consider the differences between the performance on the training/cross-validation dataset for section 1 (1.2.3) and section 2 (2.1.3). Which model is a better fit based on *just these two evaluations*? (4 marks)

#### 2.1.4 Answer
1. Section 1, The model shows perfect cross-validation training at 1.00 and 0.95 in its validation score, with a decent performance on the test set accuracy at 0.96. This implies that the performance above demonstrates strong predictive capabilities of this system. However, these findings raise concerns on the risk of overfitting while the model fits well since it may be memorizing rather than generalizing the results. In contrast, Section 2 has a more balanced performance: the cross-validation training score is 0.97, with a validation score of 0.82 and a test set accuracy of 0.83. There is a noticeable gap between the training and validation/test scores, but this does indicate that the model in Section 2 generalizes better than in Section 1, with less risk of overfitting.

Which Model is a Better Fit?
The tests show that the model in Section 2 performs more consistently across datasets and has less overfitting, making it more suitable for real-world applications. In comparison to the Section 1 model, which has a perfect training score (1.00), suggesting overfitting, the smaller difference between its training score (0.97) and validation/test scores (0.82 and 0.83, respectively) would suggest it generalizes to unknown data better. Furthermore, it is dependable in terms of performance generalization because its validation score and test set accuracy are extremely near. As seen by the mismatch observed in test_set2, Section 1 suffers from overfitting and may not generalize as well to new or other datasets, despite having a higher test accuracy of 0.96 against 0.83. For practical uses where adaptability is crucial Section 2 would seem the more reliable choice.

___

### 2.2 - Model Evaluation on Test Datasets 1 and 2 (7 Marks)

___

#### 2.2.1 - Extract selected features and targets from test datasets

The model trained in 2.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into the selected features and target vectors *(X1, y1)* and (X2,)

In [29]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# (1 marks)
y1 = test_set1['motion_state']
X1 = test_set1[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X1.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000001,7.749919e-08,9.786930,9.801661,7.727190e-10
1,0.000001,9.243000e-08,9.788127,9.802481,8.871433e-10


In [30]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
y2 = test_set2['motion_state']
X2 = test_set2[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gz_var']]
X2.head(2)

,ax_var,ay_var,az_sum,a_sum,gz_var
0,0.000002,3.020959e-07,9.791451,9.800683,1.405038e-09
1,0.000003,6.859937e-07,9.785619,9.797741,1.164381e-09


___

#### 2.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [31]:
# TODO - Print test score accuracies for test_set1 and test_set2
print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.83
test_set2 score: 0.84


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [32]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
from sklearn.metrics import confusion_matrix, classification_report

# Predict on test_set1
y1_pred = grid.predict(X1)

# Display confusion matrix for test_set1
conf_matrix1 = confusion_matrix(y1, y1_pred)
print("Confusion Matrix for test_set1:\n", conf_matrix1)

# Print classification report for test_set1
class_report1 = classification_report(y1, y1_pred)
print("\nClassification Report for test_set1:\n", class_report1)

Confusion Matrix for test_set1:
 [[229   2   0]
 [ 26 336  49]
 [  0  57  84]]

Classification Report for test_set1:
               precision    recall  f1-score   support

        rest       0.90      0.99      0.94       231
    straight       0.85      0.82      0.83       411
        turn       0.63      0.60      0.61       141

    accuracy                           0.83       783
   macro avg       0.79      0.80      0.80       783
weighted avg       0.83      0.83      0.83       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [33]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

# Predict on test_set2
y2_pred = grid.predict(X2)

# Display confusion matrix for test_set2
conf_matrix2 = confusion_matrix(y2, y2_pred)
print("Confusion Matrix for test_set2:\n", conf_matrix2)

# Print classification report for test_set2
class_report2 = classification_report(y2, y2_pred)
print("\nClassification Report for test_set2:\n", class_report2)

Confusion Matrix for test_set2:
 [[ 15   1   0]
 [  0 178  28]
 [  0  18  57]]

Classification Report for test_set2:
               precision    recall  f1-score   support

        rest       1.00      0.94      0.97        16
    straight       0.90      0.86      0.88       206
        turn       0.67      0.76      0.71        75

    accuracy                           0.84       297
   macro avg       0.86      0.85      0.85       297
weighted avg       0.85      0.84      0.84       297



___

#### 2.2.3 Questions

1. For section 2, compare between the model's performance on the training/validation (2.1) set to the the performance on the two testing sets (2.2). How does the model fit? (3 marks)
1. Compare the models' performances and fits between section 1 and section 2, and consider all three datasets. (3 marks)

#### 2.2.3 Answers
1. Comparison of Model Performance on Training/Validation (2.1) and Testing Sets (2.2):

The model may be a little overfitted during training, but it works effectively and consistently throughout validation, testing, and training. The cross-validation training score remains at 0.97 during training and validation, however it slightly drops to 0.82 during validation. This would suggest that although the model is still well-fitted, it is slightly overfitting onto the training data. Additionally, at 0.83, the test set score closely resembles the validation score, confirming the capacity to generalize.

The model results are quite consistent over the test datasets. With a balanced performance in terms of precision, recall, and F1 scores, the accuracy for test_set1 is registered at 0.83; however, the turn class has a slightly decreased F1-score of 0.61. Analysis of the confusion matrix suggests that most misclassifications are between the straight and turn classes. The model performs better on test_set 2, with an accuracy of 0.84 and improved precision and recall for all classifications. In fact, the F1-score for the turn class improves to 0.71, while the rest of the class demonstrates almost perfect performance.

2. The models in Section 1 and Section 2 exhibit contrasting behaviors in their performance and generalization capabilities across datasets.

The model in Section 1 achieves almost perfect results for both training and validation, with a test-set accuracy of 0.96, a cross-validation training score of 1.00, and a validation score of 0.95. Failure of this model on test_set2—having an accuracy of 0.25—by predicting only one class, "turn," for the whole dataset is an indication of overfitting, while the high training score suggests excellent prediction power for familiar data. While it performed well on test_set1 (accuracy: 0.92), its low generalization power is highlighted by its failure to adjust to other distributions.
Section 2, on the other hand, works better. There is a small trade-off in training accuracy for better generalization, which is indicated in the test-set accuracy of 0.83, cross-validation training score of 0.97, and validation score of 0.82. The model keeps stable accuracy and balanced metrics on datasets: test_set1 with accuracy of 0.83 and test_set2 with accuracy of 0.84. That means the model in Section 2 fits the data better and is more suitable for different types of data found in real life.

This said, Section 1 has overall better performance on known datasets, but Section 2 is the more reliable option when dealing with different and new data because of generalizing well.

____

## Section 3 - Classification Using *Group Selected Features* From Inertial Sensor Data (10 Marks)

Once again, an *SVC* model is to be trained and tuned using grid search. However, you must select which features to use from the ones provided in *cv_set*.  

____

### 3.1 Grid Search with Cross Validation (4 Marks)

___

#### 3.1.1 Extract *Group Selected* features and target from *cv_set*

**Question**
1. Please list the features that you have selected and explain *why you have chosen* these features? (4 marks)


**Answer**
1. Certain Features:
Accelerometer reading variation in the x direction is denoted by ax_var.

ay_var: Accelerometer reading variation in the y direction.

The sum of the z-directional accelerometer readings is called az_sum.

The sum of the accelerometer's x, y, and z values is known as a_sum.

Gyroscope reading variation in the x direction is denoted by gx_var.

Gyroscope reading variation in the z direction is denoted by gz_var.

Reasoning:

(1) Physical Significance- These characteristics are essential for identifying motion states (rest, straight, and turn) since they record variability and aggregate motion data from both accelerometer and gyroscope sensors. Ax_var, ay_var, gx_var, and gz_var are variance characteristics that show motion fluctuations and aid in locating transitions or steady states.
Aggregated data from sum features (az_sum, a_sum) can distinguish between different motion intensities.

(2) Dimensionality Reduction- Reducing dimensionality by simply including a subset of highly relevant features can enhance computational efficiency and generalization.

(3) Previous Performance: In Section 2, features such as ax_var, ay_var, az_sum, a_sum, and gz_var have demonstrated significant contributions to classification accuracy.

In [34]:
# TODO - Load group selected features from cv_set to X
X = cv_set[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gx_var', 'gz_var']]
X.head(2)

,ax_var,ay_var,az_sum,a_sum,gx_var,gz_var
0,8.046094e-07,1.319665e-07,9.789349,9.797418,2.365611e-09,7.446916e-10
1,1.032619e-06,1.073067e-07,9.789095,9.798815,2.242613e-09,6.983481e-10


In [35]:
# TODO - Load motion_state column (target vector) into y
y = cv_set['motion_state']

___

#### 3.1.2 - Configure and Perform Grid Search with Cross-Validation

Follow the steps from **1.2.2** and perform model training and tuning using the group selected feature set.

In [36]:
# TODO - Perform model training and tuning using `GridSearchCV`

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Define the pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),  # Default scaler
    ('classifier', SVC())          # Default classifier
])

# Define the parameter grid for grid search
param_grid = {
    'scaler': [StandardScaler(), RobustScaler(), None],  # Scalers to test
    'classifier__C': [0.01, 0.1, 1, 10, 100],           # C hyperparameter values
    'classifier__gamma': [0.001, 0.01, 0.1, 1],         # gamma hyperparameter values
    'classifier__kernel': ['rbf']                       # Radial basis function kernel
}

# Perform the grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

# Print the results of the grid search
print("Best parameters:", grid.best_params_)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))

Best parameters: {'classifier__C': 100, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf', 'scaler': StandardScaler()}
Best cross-validation score: 0.83
Test set score: 0.83


___

#### 3.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [37]:
# TODO: Print the results from the grid search

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(max(grid.cv_results_['mean_train_score'])))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.1, 'classifier__kernel': 'rbf', 'scaler': StandardScaler()}

Best cross-validation train score: 0.98
Best cross-validation validation score: 0.83
Test-set score: 0.83


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [38]:
# TODO - Display confusion matrix and print classification report

from sklearn.metrics import confusion_matrix, classification_report

# Predict on the test set
y_pred = grid.predict(X_test)

# Display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Print classification report
class_report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", class_report)

Confusion Matrix:
 [[153   2   0]
 [ 19 171  17]
 [  1  41  57]]

Classification Report:
               precision    recall  f1-score   support

        rest       0.88      0.99      0.93       155
    straight       0.80      0.83      0.81       207
        turn       0.77      0.58      0.66        99

    accuracy                           0.83       461
   macro avg       0.82      0.80      0.80       461
weighted avg       0.82      0.83      0.82       461



___

### 3.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 3.2.1 - Extract group selected features and targets from test datasets

The model trained in 3.1 is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into group selected features and target vectors *(X1, y1)* and *(X2,y2)*

In [39]:
# TODO - Load the target vector from test_set1 into y1, and the group selected features from test_set1 into X1
y1 = test_set1['motion_state']
X1 = test_set1[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gx_var', 'gz_var']]
X1.head(2)

,ax_var,ay_var,az_sum,a_sum,gx_var,gz_var
0,0.000001,7.749919e-08,9.786930,9.801661,3.655217e-09,7.727190e-10
1,0.000001,9.243000e-08,9.788127,9.802481,3.349120e-09,8.871433e-10


In [40]:
# TODO - Load the target vector from test_set2 into y2, and the group selected features from test_set2 into X2
y2 = test_set2['motion_state']
X2 = test_set2[['ax_var', 'ay_var', 'az_sum', 'a_sum', 'gx_var', 'gz_var']]
X2.head(2)

,ax_var,ay_var,az_sum,a_sum,gx_var,gz_var
0,0.000002,3.020959e-07,9.791451,9.800683,1.718706e-09,1.405038e-09
1,0.000003,6.859937e-07,9.785619,9.797741,2.179497e-09,1.164381e-09


___

#### 3.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [41]:
# TODO - Print test score accuracies for test_set1 and test_set2
print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.62
test_set2 score: 0.82


Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [42]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
from sklearn.metrics import confusion_matrix, classification_report

# Predict on test_set1
y1_pred = grid.predict(X1)

# Display confusion matrix for test_set1
conf_matrix1 = confusion_matrix(y1, y1_pred)
print("Confusion Matrix for test_set1:\n", conf_matrix1)

# Print classification report for test_set1
class_report1 = classification_report(y1, y1_pred)
print("\nClassification Report for test_set1:\n", class_report1)

Confusion Matrix for test_set1:
 [[ 92 139   0]
 [  1 323  87]
 [  0  68  73]]

Classification Report for test_set1:
               precision    recall  f1-score   support

        rest       0.99      0.40      0.57       231
    straight       0.61      0.79      0.69       411
        turn       0.46      0.52      0.49       141

    accuracy                           0.62       783
   macro avg       0.68      0.57      0.58       783
weighted avg       0.69      0.62      0.62       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [43]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

# Predict on test_set2
y2_pred = grid.predict(X2)

# Display confusion matrix for test_set2
conf_matrix2 = confusion_matrix(y2, y2_pred)
print("Confusion Matrix for test_set2:\n", conf_matrix2)

# Print classification report for test_set2
class_report2 = classification_report(y2, y2_pred)
print("\nClassification Report for test_set2:\n", class_report2)

Confusion Matrix for test_set2:
 [[ 16   0   0]
 [  0 203   3]
 [  0  51  24]]

Classification Report for test_set2:
               precision    recall  f1-score   support

        rest       1.00      1.00      1.00        16
    straight       0.80      0.99      0.88       206
        turn       0.89      0.32      0.47        75

    accuracy                           0.82       297
   macro avg       0.90      0.77      0.78       297
weighted avg       0.83      0.82      0.78       297



___

#### 3.2.3 Questions

1. Based on the observations across all three datasets, compare the performance of the model trained in this section to: (3 marks)
    1. Model trained in Section 1 using *all features*.
    1. Model trained in Section 2 using *selected features*.

2. Please proivde potential explanations for the differences observed in models' performances across the three sections. (3 marks)

#### 3.2.3 Answers
1. Selection of Features and Variability:

Section 1 (All Features) - Overfitting results from the introduction of noise and unnecessary data. Although this enhances efficiency on comparable datasets (test_set1), it makes it more difficult to generalize to datasets such as test_set2.

Section 2 (Predefined Selected Features)- Well-chosen features prevent overfitting and capture important motion properties, allowing for consistent performance across datasets.

Section 3 (Group-Selected Features)- Unbalanced classification may result from group-selected features' inability to adequately reflect variability, particularly for turn states.

2. Distribution of Datasets- (test_set1 vs. test_set2) - test_set1 probably has more in common with the training data (cv_set), but test_set2 represents various sensor or environmental conditions, making model generalization more difficult.

Model Complexity and Overfitting:
- Section 1- High complexity due to many features leads to overfitting.
- Section 2 - Balanced complexity and generalization.
Section 3 - Moderate complexity, but features selected may not generalize across all motion states.

States of Imbalanced Motion: (turn Class) - Poor recall and F1-scores are the result of the turn state's frequent underrepresentation in training and testing datasets. The group-selected characteristics in Section 3 appear especially inappropriate for this class.

____

## Section 4 - Classification Using Principal Components of Features From Inertial Sensor Data (10 Marks)

____

### 4.1 Principal Component Analysis (PCA) Search with Cross Validation (4 Marks)

___

#### 4.1.1 Extract *ALL* features and target from *cv_set*

In [44]:
# TODO - Load all features from cv_set to X
X = cv_set.drop(columns=['motion_state'])
X.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.003929,-0.093081,9.789349,-0.002649,0.001406,-0.001391,9.797418,0.022554,8.046094e-07,1.319665e-07,...,0.023871,0.000003,0.001496,0.000544,0.024992,0.000110,0.000075,0.000078,0.024997,0.000187
1,-0.000966,-0.092417,9.789095,-0.001806,0.000103,-0.001943,9.798815,0.021731,1.032619e-06,1.073067e-07,...,0.024153,0.000006,0.001865,0.000615,0.024828,0.000124,0.000098,0.000090,0.024829,0.000144


In [45]:
# TODO - Load motion_state column (target vector) into y
y = cv_set['motion_state']

___

#### 4.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning for an `SVC` model. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [46]:
# TODO - Perform model training and tuning using `GridSearchCV`
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In the `Pipeline` add an entry *'pca'* to apply `PCA`.

In [47]:
# Perform any missing imports

# TODO - Initiate the pipeline
# (1 marks)

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('classifier', SVC())
])

Similar to previous sections, define the grid search configuration (*param_grid*). Add *n_components* hyper parameter tuning for PCA, and search over the values `n_components=[4,7,11,14,16]`.  

In [48]:
# TODO - Define the parameter grid for your model
# (3 marks)
param_grid = {
    'scaler': [StandardScaler(), None],
    'pca__n_components': [4, 7, 11, 14, 16],
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__gamma': [0.001, 0.01, 0.1, 1],
    'classifier__kernel': ['rbf']
}

In [49]:
# Perform the grid search to train model and tune hyper parameters
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()), ('classifier', SVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100],
                         'classifier__gamma': [0.001, 0.01, 0.1, 1],
                         'classifier__kernel': ['rbf'],
                         'pca__n_components': [4, 7, 11, 14, 16],
                         'scaler': [StandardScaler(), None]},
             return_train_score=True, scoring='accuracy')

___

#### 4.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [50]:
# TODO: Print the results from the grid search

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation train score: {:.2f}".format(max(grid.cv_results_['mean_train_score'])))
print("Best cross-validation validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:
{'classifier__C': 100, 'classifier__gamma': 0.01, 'classifier__kernel': 'rbf', 'pca__n_components': 16, 'scaler': StandardScaler()}

Best cross-validation train score: 1.00
Best cross-validation validation score: 0.94
Test-set score: 0.94


Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [51]:
# TODO - Display confusion matrix and print classification report

from sklearn.metrics import confusion_matrix, classification_report

y_pred = grid.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

class_report = classification_report(y_test, y_pred)
print("\nClassification Report:\n", class_report)

Confusion Matrix:
 [[151   4   0]
 [ 11 195   1]
 [  0  10  89]]

Classification Report:
               precision    recall  f1-score   support

        rest       0.93      0.97      0.95       155
    straight       0.93      0.94      0.94       207
        turn       0.99      0.90      0.94        99

    accuracy                           0.94       461
   macro avg       0.95      0.94      0.94       461
weighted avg       0.94      0.94      0.94       461



___

### 4.2 - Model Evaluation on Test Datasets 1 and 2 (6 Marks)

___

#### 4.2.1 - Extract group selected features and targets from test datasets

In [52]:
# TODO - Load the target vector from test_set1 into y1, and all features from test_set1 into X1
y1 = test_set1['motion_state']
X1 = test_set1.drop(columns=['motion_state'])
X1.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,0.264465,0.089904,9.786930,-0.002305,0.000218,-0.001143,9.801661,0.025634,0.000001,7.749919e-08,...,0.024056,0.000003,0.002506,0.000845,0.024884,0.000155,0.000078,0.000069,0.024886,0.000170
1,0.264300,0.088882,9.788127,-0.001865,-0.000523,-0.001999,9.802481,0.025218,0.000001,9.243000e-08,...,0.024084,0.000007,0.002427,0.000949,0.024881,0.000201,0.000095,0.000076,0.024883,0.000201


In [53]:
# TODO - Load the target vector from test_set2 into y2, and all features from test_set2 into X2
y2 = test_set2['motion_state']
X2 = test_set2.drop(columns=['motion_state'])
X2.head(2)

,ax_sum,ay_sum,az_sum,gx_sum,gy_sum,gz_sum,a_sum,g_sum,ax_var,ay_var,...,a_min,g_min,ax_max,ay_max,az_max,gx_max,gy_max,gz_max,a_max,g_max
0,-0.046400,0.305907,9.791451,-0.002184,0.001311,-0.001901,9.800683,0.012437,0.000002,3.020959e-07,...,0.048406,0.000007,0.002575,0.002993,0.049701,0.000094,0.000115,0.000080,0.049764,0.000142
1,-0.050073,0.311947,9.785619,-0.003369,0.001898,-0.002051,9.797741,0.013805,0.000003,6.859937e-07,...,0.047696,0.000013,0.003880,0.004106,0.050477,0.000140,0.000140,0.000083,0.050763,0.000151


___

#### 4.2.2 - Visualize Results

In [54]:
# TODO - Print test score accuracies for test_set1 and test_set2
print("test_set1 score: {:.2f}".format(grid.score(X1, y1)))
print("test_set2 score: {:.2f}".format(grid.score(X2, y2)))

test_set1 score: 0.91
test_set2 score: 0.40


In [55]:
# TODO - Display confusion matrix and print classification report for (X1, y1)
from sklearn.metrics import confusion_matrix, classification_report

# Predict on test_set1
y1_pred = grid.predict(X1)

# Display confusion matrix for test_set1
conf_matrix1 = confusion_matrix(y1, y1_pred)
print("Confusion Matrix for test_set1:\n", conf_matrix1)

# Print classification report for test_set1
class_report1 = classification_report(y1, y1_pred)
print("\nClassification Report for test_set1:\n", class_report1)

Confusion Matrix for test_set1:
 [[227   4   0]
 [ 16 384  11]
 [  0  42  99]]

Classification Report for test_set1:
               precision    recall  f1-score   support

        rest       0.93      0.98      0.96       231
    straight       0.89      0.93      0.91       411
        turn       0.90      0.70      0.79       141

    accuracy                           0.91       783
   macro avg       0.91      0.87      0.89       783
weighted avg       0.91      0.91      0.90       783



Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [56]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

# Predict on test_set2
y2_pred = grid.predict(X2)

# Display confusion matrix for test_set2
conf_matrix2 = confusion_matrix(y2, y2_pred)
print("Confusion Matrix for test_set2:\n", conf_matrix2)

# Print classification report for test_set2
class_report2 = classification_report(y2, y2_pred)
print("\nClassification Report for test_set2:\n", class_report2)

Confusion Matrix for test_set2:
 [[  0  16   0]
 [  0  45 161]
 [  0   0  75]]

Classification Report for test_set2:
               precision    recall  f1-score   support

        rest       0.00      0.00      0.00        16
    straight       0.74      0.22      0.34       206
        turn       0.32      1.00      0.48        75

    accuracy                           0.40       297
   macro avg       0.35      0.41      0.27       297
weighted avg       0.59      0.40      0.36       297



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


___

#### 4.2.3 - Questions

1. Based on the observations across all three datasets, compare the performance of the model with PCA, trained in this section to: (4 marks)
    * Model trained in Section 1 using *all features*.
    * Model trained in Section 2 using *selected features*.
    * Model trained in Section 3 using *group selected features*

2. Please provide potential explanations for the differences observed in models' performances across between section 1 and 4. (2 marks)

#### 4.2.3 - Answers

____

## Section 5 - Classification Using *Pre Selected Features* Derived From Inertial Sensor Data (10 Marks)

___

Now, a second classifer is to be trained and tuned using grid search. The **selected features from section 2** must be used. You may choose from one of the algorithms discussed that class listed below.
1. Random Forest
1. K-Nearest Neighbors
1. Gradient Boosting Machines
1. Neural Networks

____

### 5.1 Grid Search with Cross Validation (6 Marks)

___

#### 5.1.1 Extract *Selected* features and target from *cv_set*

**Questions**
1. Please list the ML model that you have selected . And explain *why you have chosen* this model? (2 marks)

**Answer**
1.

In [ ]:
# TODO - Load group selected features from cv_set to X
# X =
X.head(2)

In [ ]:
# TODO - Load motion_state column (target vector) into y
# y =

___

#### 5.1.2 - Configure and Perform Grid Search with Cross-Validation

Similar to the steps from **1.2.2** and perform model training and tuning using for the model selected. Perform the `train_test_split`, initiate a `Pipeline`, then define the *Parameters* for `GridSearchCV`. Finally, perfrom the grid search.

In [ ]:
# TODO - Perform train/test split

In [ ]:
# Perform any missing imports

# TODO - Initiate the pipeline
# (1 mark)
# pipe =

In [ ]:
# TODO - Define the parameter grid for your model
# (3 marks)
# param_grid =

In [ ]:
# TODO - Perform the grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid.fit(X_train, y_train)

___

#### 5.1.3 - Visualize Grid Search Results

Print the results from the grid search. The following outputs must be printed.
* Best parameter set
* Best cross-validation train score
* Best cross-validation test score
* Test set accuracy

In [ ]:
# TODO: Print the results from the grid search

#print("Best params:\n{}\n".format(???))
#print("Best cross-validation train score: {:.2f}".format(???))
#print("Best cross-validation validation score: {:.2f}".format(???))
#print("Test-set score: {:.2f}".format(???)))

Display the confusion matrix and print the classification report for the trained model using the *test* split.

In [ ]:
# TODO - Display confusion matrix and print classification report

___

### 5.2 - Model Evaluation on Test Datasets 1 and 2 (4 Marks)

___

#### 5.2.1 - Extract selected features and targets from test datasets

The model trained in 5.1, is to be tested on the independent datasets, that is test_set1 and test_set2.
Seperate the testsets into features and target vectors *(X1, y1)* and *(X2,y2)*

In [ ]:
# TODO - Load the target vector from test_set1 into y1, and the selected features from test_set1 into X1
# y1 =
# X1 =
X1.head(2)

In [ ]:
# TODO - Load the target vector from test_set2 into y2, and the selected features from test_set2 into X2
# y2 =
# X2 =
X2.head(2)

___

#### 5.2.2 - Visualize Results

For each test dataset, print the test set accuracy score.

In [ ]:
# TODO - Print test score accuracies for test_set1 and test_set2
#print("test_set1 score: {:.2f}".format(???))
#print("test_set2 score: {:.2f}".format(???))

Display the confusion matrix and print the classification for test_set 1 *(X1, y1)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X1, y1)

Display the confusion matrix and print the classification for test_set 2 *(X2, y2)*

In [ ]:
# TODO - Display confusion matrix and print classification report for (X2, y2)

___

#### 5.2.3 Question

1. For the model trainied in this section, compare its performance to the performance of SVC which was also trained with the same selected features (section 2). Which model fits better? Explain causes for any observed discrepancies? (4 marks)

#### 5.2.3 - Answer
1.


____